## Описание

Домашнее задание номер 1. Про трансфер лернинг.

Идея заключается в том, что часто учить большую сеть с нуля для новой задачи не хочется. При этом, первые слои во многих задачах детектируют одни и те же фичи. Значит для решения своей задачи можно взять сетку обученную на похожем датасете, взять ее веса, кроме последнего полносвязного слоя (или нескольких), и дообучить последний слой (или несколько) для своей задачи.

В данном задании, в папке ./weighsts лежат веса предобученной сетки для классификации цифр 4 и 8. Предлагается написать код, который переучит эту сетку на любые две другие цифры и сравнить по времени с обучением сети с нуля.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
from IPython import display

%matplotlib inline

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

### Загружаем данные, пока выделяем цифры 4 и 8. 

In [ ]:
import pandas as pd

data = pd.read_csv("../../6.Intro_to_NN/data/train.csv")

x_all = data[data.columns[1:]].values
y_all = data[data.columns[0]].values

In [ ]:
mask = np.logical_or(y_all == 4, y_all == 8)

In [ ]:
x,y = x_all[mask], y_all[mask]

In [ ]:
y = (y == 4).astype(int)

Сетка предобучена на картинках ужатых в 14 на 14 пикселей.

In [ ]:
INPUT_SHAPE = (14, 14)

In [ ]:
x = x.reshape(-1, 28,28)[:, ::2,::2]

In [ ]:
from sklearn.model_selection import train_test_split

x_tr, x_te, y_tr, y_te = train_test_split(x, y)

Будьте внимательны, если вы используете GPU, все тензоры должны быть так или иначе отправленны туда. Например с помощью метода .cuda()

In [ ]:
x_tr = torch.from_numpy(x_tr.astype(np.float32)).cuda()
x_te = torch.from_numpy(x_te.astype(np.float32)).cuda()
y_tr = torch.from_numpy(y_tr.astype(int)).cuda()
y_te = torch.from_numpy(y_te.astype(int)).cuda()

In [ ]:
x_tr = x_tr.reshape(x_tr.shape[0], 1, x_tr.shape[1], x_tr.shape[2])
x_te = x_te.reshape(x_te.shape[0], 1, x_te.shape[1], x_te.shape[2])

Код для загрузки батчей

In [ ]:
def iterate_minibatches(x, y, batchsize, shuffle=True):
    if shuffle:
        indices = np.arange(x.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, x.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield x[excerpt], y[excerpt]

## Инициализируем сетку

Чтобы это произошло на GPU, в инициализации добавляем .cuda()

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        self.conv_1 = nn.Conv2d(1, 8, (3,3))
        self.max_p_1 = nn.MaxPool2d((2,2))
        self.conv_2 = nn.Conv2d(8, 16, (3,3))
        self.max_p_2 = nn.MaxPool2d((2,2))
        self.linear = nn.Linear(64, 2)
       
        
    def forward(self, x):
        x = self.conv_1(x)
        x = F.tanh(x)
        x = self.max_p_1(x)
        
        #print(x.shape)
        
        x = self.conv_2(x)
        x = F.tanh(x)
        x = self.max_p_2(x)
        
        #print(x.shape)
        x = x.reshape(-1, 64)
        x = self.linear(x)
        x = F.softmax(x, dim = 1)
        return x


net = Net().cuda()
print(net)

## Загрузка параметров в сеть 
Загрузить нужно все параметры кроме последнего полносвязного слоя. Его нужно оставить таким, какой был при инициализации.

Веса сеток в торче загружаются и сохраняются с помощью так называемого state_dict. Это словарь в котором ключи - названия весов и слоев сетки, а переменные это веса. 
Для хранения, загрузки и сохранения таких словарей используются функции:
- torch.load() - загружает state_dict из файла
- torch.save() - сохраняет state_dict в файл
И методы самой сетки
- net.load_state_dict() - загружает веса в сетку
- net.state_dict() - вызвращает веса сетки.

Чтобы подгрузить не все веса в сеть, нужно сначала загрузить словарь из файла, затем удалить часть весов из него, потом воспользоваться .load_state_dict(), обратив внимание на аргумент strict. 

Загрузите нужные веса в сеть:

In [ ]:
* YOUR CODE HERE * 

## Новая задача для сетки

Теперь выделите из датасета любые другие цифры или сделайте классификацию всех цифр сразу. 

In [ ]:
x_tr_new = * YOUR CODE HERE * 
x_te_new = * YOUR CODE HERE * 
y_tr_new = * YOUR CODE HERE * 
y_te_new = * YOUR CODE HERE * 

## Заморозка градиенов

Мы хотим учить только последний слой сетки и не испортить веса во всех остальных. Значит нужно для всех прдыдущих слоев проставить requires_grad = False. Тогда во время оптимизации градиенты для них не будут считаться, а значит не будет и не нужных изменений.

Обратиться к слоям и весам сетки можно разными способами. Например:
- net.parameters()
- net.*layer_name*.weight и net.*layer_name*.bias

Заморозте все слои инициализированной сетки, кроме последнего

In [ ]:
* YOUR CODE HERE *

## Обучение

Ниже стандартный код для обучения. Дообучите сетку на новой задаче.

In [ ]:
from torch.optim import Adam, SGD

In [ ]:
criterion = nn.CrossEntropyLoss()
sgd = SGD(net.parameters(), lr = 0.0001)

In [ ]:
train_losses = []
val_losses = []

In [ ]:
epochs = 50

In [ ]:
for epoch in range(epochs):
    batch_losses = []
    for i, (batch_x, batch_y) in enumerate(iterate_minibatches(x_tr[:], y_tr[:], 20)):       
        out = net(batch_x)
        target = batch_y

        sgd.zero_grad()
        loss = criterion(out, target)

        loss.backward()
        sgd.step()
        
        batch_losses.append(loss.item())

    train_losses.append(np.mean(batch_losses))
    
    val_losses.append(criterion(net(x_te), y_te).item())

    
    display.clear_output(wait=True)
    plt.plot(train_losses, label='train')
    plt.plot(val_losses, label='val')
    plt.legend()
    plt.show()

## Ранняя остановка

Обучение имеет смысл проводить пока ошибка на тестовом датасете не перестала падать. Критерием такой остановки может служить то, что минимальная ошибка на тестовом сете была достигнута не на последних К эпохах, где К некоторый параметр порядка 5-10.

Чтобы удобно учить сетки реализуйте функцию, которая принимает на вход сетку, оптимизатор и датасет, максимальное кол-во эпох, размер батча и длину окна для ранней остановки, затем обучает сеть и возвращает сетку, оптимизатор и массивы с train и test ошибками для каждой эпохи и флажок о том произошел ли early stop.

In [ ]:
def train_early_stop(net, 
                     optimizer,
                     x_tr, x_te, y_tr, y_te,
                     max_epochs,
                     batch_size,
                     early_stop_window):
    *YOUR CODE HERE*
    
    return net, optimizer, train_losses, val_losses, early_stop_happend

## Сравнение сеток

Теперь обучите сетку инициализированную случайнымми весами и сетку в которой первые слои инициализированы из обученной сетки и нужные градиенты замороженны. Замерьте время обучения. Для этого можно использовать cell magic - особые команды для юпитера, - вам нужна команда %time или %%time